# Data Cleaning & EDA

This notebook prepares the Spotify dataset for trend analysis.  
Steps:
1. Load merged dataset
2. Handle missing values & duplicates
3. Clean categorical values (genres, artists, etc.)
4. Analyze:
   - Top artists
   - Top songs
   - Genre trends
   - Seasonal/Yearly patterns

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Display settings
pd.set_option("display.max_columns", 50)

In [ ]:
df = pd.read_csv("data/processed/spotify_merged.csv")
print(df.shape)
df.head()

In [ ]:
# Drop duplicates
df = df.drop_duplicates()

# Handle missing values
missing = df.isnull().sum()
print("Missing values:\n", missing[missing > 0])

# Example: fill popularity with mean
if "popularity" in df.columns:
    df["popularity"] = df["popularity"].fillna(df["popularity"].mean())

# Drop rows where track name or artist is missing
df = df.dropna(subset=["track_name", "artist_names"])

In [ ]:
# Ensure date column is datetime
df["date"] = pd.to_datetime(df["date"], errors="coerce")

# Extract year, month
df["year"] = df["date"].dt.year
df["month"] = df["date"].dt.month

In [ ]:
top_artists = df.groupby("artist_names")["streams"].sum().reset_index()
top_artists = top_artists.sort_values(by="streams", ascending=False).head(10)

plt.figure(figsize=(10,6))
sns.barplot(data=top_artists, x="streams", y="artist_names", palette="viridis")
plt.title("Top 10 Artists by Total Streams")
plt.show()

In [ ]:
top_songs = df.groupby("track_name")["streams"].sum().reset_index()
top_songs = top_songs.sort_values(by="streams", ascending=False).head(10)

fig = px.bar(top_songs, x="streams", y="track_name", orientation="h",
             title="Top 10 Songs by Streams", color="streams")
fig.show()

In [ ]:
if "genre" in df.columns:
    genre_trends = df.groupby(["year", "genre"])["streams"].sum().reset_index()
    
    fig = px.line(genre_trends, x="year", y="streams", color="genre",
                  title="Genre Popularity Over Years")
    fig.show()


In [ ]:
seasonal = df.groupby("month")["streams"].sum().reset_index()

plt.figure(figsize=(10,6))
sns.lineplot(data=seasonal, x="month", y="streams", marker="o")
plt.title("Seasonal Music Trends (by Month)")
plt.xticks(range(1,13))
plt.show()

In [ ]:
if "popularity" in df.columns:
    plt.figure(figsize=(8,5))
    sns.histplot(df["popularity"], bins=30, kde=True, color="green")
    plt.title("Distribution of Song Popularity")
    plt.xlabel("Popularity Score (0-100)")
    plt.show()

In [ ]:
audio_features = ["danceability", "energy", "valence", "tempo", "acousticness", "instrumentalness", "liveness", "speechiness"]

features_df = df[audio_features].dropna()

plt.figure(figsize=(10,7))
sns.heatmap(features_df.corr(), annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Correlation Between Audio Features")
plt.show()


In [ ]:
yearly_features = df.groupby("year")[audio_features].mean().reset_index()

fig = px.line(yearly_features, x="year", y="danceability", title="Average Danceability Over Years")
fig.show()

fig = px.line(yearly_features, x="year", y="energy", title="Average Energy Over Years")
fig.show()

fig = px.line(yearly_features, x="year", y="tempo", title="Average Tempo Over Years")
fig.show()


In [ ]:
top_countries = df.groupby("region")["streams"].sum().reset_index().sort_values(by="streams", ascending=False).head(10)

plt.figure(figsize=(10,6))
sns.barplot(data=top_countries, x="streams", y="region", palette="magma")
plt.title("Top 10 Countries by Total Streams")
plt.show()

In [ ]:
artist_trends = df.groupby(["year", "artist_names"])["streams"].sum().reset_index()

top5_artists = artist_trends.groupby("artist_names")["streams"].sum().nlargest(5).index
filtered = artist_trends[artist_trends["artist_names"].isin(top5_artists)]

fig = px.line(filtered, x="year", y="streams", color="artist_names",
              title="Top 5 Artists Trends Over Years")
fig.show()

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

text = " ".join(df["track_name"].dropna().astype(str))
wordcloud = WordCloud(width=800, height=400, background_color="black").generate(text)

plt.figure(figsize=(12,6))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.title("Most Common Words in Song Titles")
plt.show()

In [ ]:
# Save key summaries for reporting
top_artists.to_csv("data/processed/top_artists.csv", index=False)
top_songs.to_csv("data/processed/top_songs.csv", index=False)
top_countries.to_csv("data/processed/top_countries.csv", index=False)
yearly_features.to_csv("data/processed/yearly_features.csv", index=False)

In [ ]:
df.to_csv("data/processed/spotify_cleaned.csv", index=False)

# Wrap-Up

This notebook:
- Cleaned and preprocessed the Spotify dataset
- Found top artists & songs
- Analyzed genres over time
- Explored seasonal music patterns

Next step:  
📓 `03_dashboard_powerbi.pbix` → Create a Power BI Dashboard for interactive insights.